(spreadsheets)=
# Spreadsheets

## Introduction

This chapter will show you how to work with spreadsheets, for example Microsoft Excel files, in Python. We already saw how to import csv (and tsv) files in {ref}`data-import`. In this chapter we will introduce you to tools for working with data in Excel spreadsheets and Google Sheets.

If you or your collaborators are using spreadsheets for organising data that will be ingested by an analytical tool like Python, we recommend reading the paper "Data Organization in Spreadsheets" by Karl Broman and Kara Woo {cite}`broman2018data`. The best practices presented in this paper will save you much headache down the line when you import the data from a spreadsheet into Python to analyse and visualise. (For spreadsheets that are meant to be read by humans, we recommend the [good practice tables](https://github.com/best-practice-and-impact/gptables) package.)

In [ ]:
# remove cell
import matplotlib_inline.backend_inline
import matplotlib.pyplot as plt

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

### Prerequisites

You will need to install the **pandas** package for this chapter. You will also need to install the **openpyxl** package by running `pip install openpyxl` in the terminal.


## Getting Started

### Reading Excel (and Similar) Files

**pandas** can read in xls, xlsx, xlsm, xlsb, odf, ods, and odt files from your local filesystem or from a URL. It also supports an option to read a single sheet or a list of sheets.

To show how this works, we'll work with an example spreadsheet called "students.xlsx". The figure below shows what the spreadsheet looks like.

![A look at the students spreadsheet in Excel. The spreadsheet contains information on 6 students, their ID, full name, favourite food, meal plan, and age.](https://github.com/hadley/r4ds/raw/main/images/import-spreadsheets-students.png)

The first argument to `pd.read_excel()` is the path to the file to read. If you have downloaded the [file]() onto your computer and put it in a subfolder called "data" then you would want to use the path "data/students.xlsx" but we can also load it directly from the URL.

In [ ]:
import pandas as pd
import numpy as np

students = pd.read_excel(
    "https://github.com/aeturrell/python4DS/raw/main/data/students.xlsx"
)
students

We have six students in the data and five variables on each student. However there are a few things we might want to address in this dataset:

- The column names are all over the place. You can provide column names that follow a consistent format; we recommend `snake_case` using the `names` argument.


In [ ]:
pd.read_excel(
    "https://github.com/aeturrell/python4DS/raw/main/data/students.xlsx",
    names=["student_id", "full_name", "favourite_food", "meal_plan", "age"],
)


- `age` is read in as a column of objects, but it really should be numeric. Just like with `read_csv()`, you can supply a `dtype` argument to `read_excel()` and specify the data types for the columns of data you read in. Your options include `"boolean"`, `"int"`, `"float"`, `"datetime"`, `"string"`, and more. But we can see right away that this isn't going to work with the "age" column as it mixes numbers and text: so we first need to map its text to numbers.

In [ ]:
students = pd.read_excel(
    "data/students.xlsx",
    names=["student_id", "full_name", "favourite_food", "meal_plan", "age"],
)
students["age"] = students["age"].replace("five", 5)
students

Okay, now we can apply the data types.

In [ ]:
students = students.astype(
    {
        "student_id": "Int64",
        "full_name": "string",
        "favourite_food": "string",
        "meal_plan": "category",
        "age": "Int64",
    }
)
students.info()

It took multiple steps and trial-and-error to load the data in exactly the format we want, and this is not unexpected. Data science is an iterative process. There is no way to know exactly what the data will look like until you load it and take a look at it. The general pattern we used is load the data, take a peek, make adjustments to your code, load it again, and repeat until you're happy with the result.

### Reading Individual Sheets

An important feature that distinguishes spreadsheets from flat files is the notion of multiple sheets. The figure below shows an Excel spreadsheet with multiple sheets. The data come from the **palmerpenguins** dataset {ref}`horst2020palmerpenguins`. Each sheet contains information on penguins from a different island where data were collected.

![A look at the penguins spreadsheet in Excel. The spreadsheet contains has three sheets: Torgersen Island, Biscoe Island, and Dream Island.](https://github.com/hadley/r4ds/raw/main/images/import-spreadsheets-penguins-islands.png)

You can read a single sheet using the following command (so as not to show the whole file, we'll use `.head()` to just show the first 5 rows):

In [ ]:
pd.read_excel(
    "https://github.com/aeturrell/python4DS/raw/main/data/penguins.xlsx",
    sheet_name="Torgersen Island",
).head()

Now this relies on us knowing the names of the sheets in advance. There will be situations where you can to read in data without peeking into the Excel spreadsheet. To read all sheets in, use `sheet_name=None`. The object that's created is a dictionary with key value pairs that are sheet names and data frames respectively. Let's look at the second key value pair (note that we have to convert the keys() and values() objects to list to then retrieve the second element of each using a subscript, ie `list(dictionary.keys())[<element number>]`).

To give a sense of how this works, let's first print all of the retrieved keys:

In [ ]:
penguins_dict = pd.read_excel(
    "https://github.com/aeturrell/python4DS/raw/main/data/penguins.xlsx",
    sheet_name=None,
)
print([x for x in penguins_dict.keys()])

Now let's show the second entry data frame

In [ ]:
print(list(penguins_dict.keys())[1])
list(penguins_dict.values())[1].head()

What we really want is these three *consistent* datasets to be in the *same* single dataframe. For this, we can use the `pd.concat` function. This concatenates any given iterable of data frames.

In [ ]:
penguins = pd.concat(penguins_dict.values(), axis=0)
penguins

### Reading Part of a Sheet

Since many use Excel spreadsheets for presentation as well as for data storage, it's quite common to find cell entries in a spreadsheet that are not part of the data you want to read in.
